In [1]:
%load_ext autoreload
%env CUDA_VISIBLE_DEVICES = 2 python benchmark.ipynb

env: CUDA_VISIBLE_DEVICES=2 python benchmark.ipynb


In [2]:
import pandas as pd
import torch.nn as nn

from benchmark import tracker

%autoreload 2
pd.set_option("display.precision", 2)

In [3]:
device = 'cuda'
device

'cuda'

In [4]:
shape = (1, 64, 128, 128)

module_collection = {'conv_64_512_1x1': nn.Conv2d(kernel_size=1, in_channels=64, out_channels=512), 
                     'conv_64_512_3x3': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512), 
                     'bottleneck_64_512_3x3': nn.Sequential(*[nn.Conv2d(kernel_size=1, in_channels=64, out_channels=32), 
                                                             nn.Conv2d(kernel_size=3, in_channels=32, out_channels=512),
                                                              ]),
                      'conv_64_512_3x3_g2': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=2),
                      'conv_64_512_3x3_g8': nn.Conv2d(kernel_size=7, in_channels=64, out_channels=512, groups=8),
                      'conv_padding': nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512, padding=1),
                      'conv_nn_Padding': nn.Sequential(nn.ZeroPad2d(1),
                                                       nn.Conv2d(kernel_size=3, in_channels=64, out_channels=512))
                    }

In [5]:
# contiguous
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters', 'time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, memory_unit='KB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.41,0.06,0.37,0.03,33786.37,131.07,545259520.0,33280.0,1414.21
conv_64_512_3x3,0.8,0.04,1.23,0.03,36835.33,1179.65,4690151424.0,295424.0,776.3
bottleneck_64_512_3x3,0.8,0.04,1.47,0.03,37294.08,2686.98,2383218688.0,150048.0,831.69
conv_64_512_3x3_g2,1.9,0.05,2.62,0.02,34849.28,3211.26,11956733952.0,803328.0,1593.92
conv_64_512_3x3_g8,0.65,0.03,1.43,0.04,32983.04,802.82,2994898944.0,201216.0,816.18
conv_padding,0.74,0.09,0.93,0.08,36835.33,91357.18,4840226816.0,295424.0,717.65
conv_nn_Padding,0.83,0.03,1.28,0.03,41291.78,1179.65,4840226816.0,295424.0,770.21


In [6]:
# channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, channels_last=True, memory_unit='MB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.45,0.05,0.29,0.06,38.24,35.65,545259520.0,33280.0,378.18
conv_64_512_3x3,0.98,0.03,1.33,0.05,39.03,35.65,4690151424.0,295424.0,877.11
bottleneck_64_512_3x3,0.83,0.06,2.35,0.31,41.57,35.65,2383218688.0,150048.0,1228.21
conv_64_512_3x3_g2,1.86,0.04,13.4,0.09,39.24,41.11,11956733952.0,803328.0,5325.55
conv_64_512_3x3_g8,1.04,0.03,12.03,0.33,37.37,36.45,2994898944.0,201216.0,4542.15
conv_padding,0.95,0.03,2.0,0.05,40.04,35.65,4840226816.0,295424.0,1086.98
conv_nn_Padding,1.03,0.05,1.28,0.09,44.3,35.65,4840226816.0,295424.0,908.99


In [7]:
# amp
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, amp=True, memory_unit='GB')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.4,0.07,0.38,0.06,0.04,0.04,545259520.0,33280.0,392.37
conv_64_512_3x3,0.49,0.06,0.45,0.05,0.04,0.04,4690151424.0,295424.0,434.33
bottleneck_64_512_3x3,0.68,0.04,0.89,0.11,0.04,0.04,2383218688.0,150048.0,708.26
conv_64_512_3x3_g2,10.59,0.06,11.93,0.15,0.04,0.04,11956733952.0,803328.0,7787.11
conv_64_512_3x3_g8,2.5,0.03,9.9,0.07,0.04,0.04,2994898944.0,201216.0,4317.09
conv_padding,0.48,0.03,0.6,0.07,0.04,0.04,4840226816.0,295424.0,479.33
conv_nn_Padding,0.53,0.05,0.58,0.17,0.04,0.04,4840226816.0,295424.0,520.21


In [8]:
# amp + channels last
module_collection_stats = pd.DataFrame(index=module_collection.keys(), 
                                               columns=['forward time mean(ms)', 'forward time std(ms)',
                                                        'backward time mean(ms)', 'backward time std(ms)',
                                                        'forward memory','backward memory',
                                                        'macs', 'parameters','time total(ms)'])
        
for module_name, module_value in module_collection.items():
    module_collection_stats.loc[module_name] = tracker(module_value, inputs=shape, device=device, track_backward=True, channels_last=True, amp=True, memory_unit='B')

module_collection_stats

,forward time mean(ms),forward time std(ms),backward time mean(ms),backward time std(ms),forward memory,backward memory,macs,parameters,time total(ms)
conv_64_512_1x1,0.35,0.03,0.33,0.07,37980672.0,35654144.0,545259520.0,33280.0,352.52
conv_64_512_3x3,0.47,0.04,0.58,0.18,38805504.0,35654144.0,4690151424.0,295424.0,464.87
bottleneck_64_512_3x3,0.69,0.05,1.06,0.24,39493120.0,35654144.0,2383218688.0,150048.0,751.15
conv_64_512_3x3_g2,10.56,0.05,11.96,0.04,38068736.0,41105920.0,11956733952.0,803328.0,7787.04
conv_64_512_3x3_g8,2.51,0.03,9.91,0.05,36202496.0,36454400.0,2994898944.0,201216.0,4312.23
conv_padding,0.46,0.03,0.57,0.07,39845888.0,35654144.0,4840226816.0,295424.0,458.01
conv_nn_Padding,0.52,0.05,0.59,0.16,44304384.0,35654144.0,4840226816.0,295424.0,516.14
